In [78]:
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import math

ORBIT_DATA_DIR  = Path(r"E:\interim\step2_normalized_test")
OB_MEET_TIME    = Path(r"E:\tables\orbit_meet_time_condition_test.csv")
OUTPUT_DIR      = Path(r"E:\tables")

print("orbit data directory =", {ORBIT_DATA_DIR})
print("Orbit_meet_time_condion.csv path=", {OB_MEET_TIME})
print("Directory of output", {OUTPUT_DIR})

a = 6378137                                                         # the long radius of the Earth
b = 6356752.314                                                     # the short radius of the Earth

e = math.sqrt((a*a - b*b)/(a*a))                                    # eccentricity of the Earth

df_obmt = pd.read_csv(OB_MEET_TIME)
print(df_obmt)
length_omt = len(df_obmt)

list1 = [[] for k in range(length_omt)]



orbit data directory = {WindowsPath('E:/interim/step2_normalized_test')}
Orbit_meet_time_condion.csv path= {WindowsPath('E:/tables/orbit_meet_time_condition_test.csv')}
Directory of output {WindowsPath('E:/tables')}
    Unnamed: 0                           orbit_file  eq_id  \
0            0  DMT_N1_1132_00197.1_step1_step2.csv    383   
1            1  DMT_N1_1132_00198.1_step1_step2.csv    383   
2            2  DMT_N1_1132_00582.1_step1_step2.csv    434   
3            3  DMT_N1_1132_00583.1_step1_step2.csv    434   
4            4  DMT_N1_1132_00584.1_step1_step2.csv    434   
5            5  DMT_N1_1132_00591.1_step1_step2.csv    435   
6            6  DMT_N1_1132_00592.1_step1_step2.csv    435   
7            7  DMT_N1_1132_00593.1_step1_step2.csv    435   
8            8  DMT_N1_1132_00595.1_step1_step2.csv    436   
9            9  DMT_N1_1132_00596.1_step1_step2.csv    436   
10          10  DMT_N1_1132_00597.1_step1_step2.csv    436   
11          11  DMT_N1_1132_00608.1_step

In [79]:
for i in range(length_omt):
    list2 = []
    orbit_file_name = df_obmt["orbit_file"].iloc[i]
    
    orbit_data_path = ORBIT_DATA_DIR / orbit_file_name

    df_obdata = pd.read_csv(orbit_data_path)

    
    for j in range(len(df_obdata)):
        lat1 = df_obmt["eq_lat"].iloc[i] * math.pi/180
        lon1 = df_obmt["eq_lon"].iloc[i] * math.pi/180

        lat2 = df_obdata["lat"].iloc[j] * math.pi/180
        lon2 = df_obdata["lon"].iloc[j] * math.pi/180

        dif_lat = lat2 - lat1
        dif_lon = lon2 - lon1
        if dif_lon > math.pi:
            dif_lon -= 2*math.pi
        elif dif_lon < -math.pi:
            dif_lon += 2*math.pi
        
        P = (lat1+lat2) / 2                                         # 両点緯度の平均値
        W = math.sqrt((1-e*e * math.sin(P) * math.sin(P)))
        M = (a*(1 - e*e)) / (W * W * W)
        N = a / W

        dist = math.sqrt(dif_lat*dif_lat*M*M + dif_lon*dif_lon*N*N*math.cos(P)*math.cos(P))
        dist = dist / 1000                                          # 単位[km] => [m]

        if dist < 330:
            beforeq = pd.to_datetime(df_obmt["4hour_before"].iloc[i]).tz_localize(None)
            print("beforeq", beforeq, "type=", type(beforeq))
            starteq = pd.to_datetime(df_obmt["datetime"].iloc[i]).tz_localize(None)
            print("starteq", starteq, "type=", type(starteq))
            s1 = pd.to_datetime(df_obdata["datetime"].iloc[j])
            print("s1", s1, "type=", type(s1))

            if beforeq <= s1 <= starteq:
                list2.append(orbit_file_name)
                break
    
    print(list2)
    list1[i] = list2


[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
beforeq 2004-08-12 08:55:14.980000 type= <class 'pandas._libs.tslibs.timestamps.Timestamp'>
starteq 2004-08-12 12:55:14.980000 type= <class 'pandas._libs.tslibs.timestamps.Timestamp'>
s1 2004-08-12 12:47:51.523000064 type= <class 'pandas._libs.tslibs.timestamps.Timestamp'>
['DMT_N1_1132_00597.1_step1_step2.csv']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [ ]:
print(list1)
data = pd.DataFrame(list1)
output_one = pd.concat([data, df_obmt], axis=1)
print(output_one)


[[], [], [], [], [], [], [], [], [], [], ['DMT_N1_1132_00597.1_step1_step2.csv'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
                                      0  Unnamed: 0  \
0                                  None           0   
1                                  None           1   
2                                  None           2   
3                                  None           3   
4                                  None           4   
5                                  None           5   
6                                  None           6   
7                                  None           7   
8                                  None           8   
9                                  None           9   
10  DMT_N1_1132_00597.1_step1_step2.csv          10   
11                                 None          11   
12                                 None          12   
13                                 None          13   
14                               